# Import related libs

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import mymodule as mm

In [2]:
# read the dataset
#source: https://www.kaggle.com/gsutters/the-human-freedom-index#hfi_cc_2018.csv
df = pd.read_excel("https://github.com/lifegivesyoulemons/udata/blob/master/SuperStoreUS_2015.xlsx?raw=true", sheet_name = 'Orders', header=0)
df = df.drop(columns=['Row ID', 'Postal Code', 'Order Date', 'Ship Date', 'Order ID', 'Product Name', 'Customer ID', 'Customer Name'])

# Explore data

In [3]:
df.iloc[50:60]

,Order Priority,Discount,Unit Price,Shipping Cost,Ship Mode,Customer Segment,Product Category,Product Sub-Category,Product Container,Product Base Margin,Country,Region,State or Province,City,Profit,Quantity ordered new,Sales
50,High,0.07,8.34,1.43,Regular Air,Home Office,Office Supplies,Paper,Wrap Bag,0.35,United States,South,Louisiana,Terrytown,-190.6800,16.0,132.08
51,High,0.09,4.98,6.07,Regular Air,Home Office,Office Supplies,Paper,Small Box,0.36,United States,South,Louisiana,Terrytown,325.3980,9.0,45.34
52,Medium,0.04,12.98,3.14,Express Air,Corporate,Office Supplies,"Scissors, Rulers and Trimmers",Small Pack,0.60,United States,South,Louisiana,Terrytown,22.8180,16.0,216.04
53,Not Specified,0.04,160.98,30.00,Delivery Truck,Home Office,Furniture,Chairs & Chairmats,Jumbo Drum,0.62,United States,Central,Illinois,Chicago,116.1000,37.0,6276.34
54,Not Specified,0.01,17.98,4.00,Regular Air,Home Office,Technology,Computer Peripherals,Small Box,0.79,United States,Central,Illinois,Chicago,-87.9600,146.0,2664.40
55,Not Specified,0.04,160.98,30.00,Delivery Truck,Home Office,Furniture,Chairs & Chairmats,Jumbo Drum,0.62,United States,Central,Kansas,Manhattan,255.4200,9.0,1526.68
56,Not Specified,0.06,115.99,8.99,Regular Air,Home Office,Technology,Telephones and Communication,Small Box,0.58,United States,Central,Kansas,Manhattan,685.6146,20.0,1952.56
57,Medium,0.10,19.98,4.00,Regular Air,Consumer,Technology,Computer Peripherals,Small Box,0.68,United States,East,Maine,Biddeford,-16.2000,16.0,303.59
58,Medium,0.04,300.98,54.92,Delivery Truck,Consumer,Furniture,Bookcases,Jumbo Box,0.55,United States,East,Massachusetts,Boston,2023.7500,31.0,9459.94
59,Medium,0.10,19.98,4.00,Regular Air,Consumer,Technology,Computer Peripherals,Small Box,0.68,United States,East,Massachusetts,Boston,-20.2500,65.0,1233.32


In [4]:
#dataframe info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1952 entries, 0 to 1951
Data columns (total 17 columns):
Order Priority          1952 non-null object
Discount                1945 non-null float64
Unit Price              1944 non-null float64
Shipping Cost           1946 non-null float64
Ship Mode               1947 non-null object
Customer Segment        1945 non-null object
Product Category        1951 non-null object
Product Sub-Category    1952 non-null object
Product Container       1941 non-null object
Product Base Margin     1936 non-null float64
Country                 1952 non-null object
Region                  1952 non-null object
State or Province       1952 non-null object
City                    1952 non-null object
Profit                  1945 non-null float64
Quantity ordered new    1925 non-null float64
Sales                   1936 non-null float64
dtypes: float64(7), object(10)
memory usage: 259.3+ KB


In [5]:
# describe dataframe 
df.describe()

,Discount,Unit Price,Shipping Cost,Product Base Margin,Profit,Quantity ordered new,Sales
count,1945.000000,1944.000000,1946.00000,1936.000000,1945.000000,1925.000000,1936.000000
mean,0.048992,109.114897,12.99093,0.515186,113.503320,12.944416,968.964592
std,0.031404,394.209477,17.43460,0.137055,1142.292351,13.933668,2491.166164
min,0.000000,1.140000,0.49000,0.350000,-16476.838000,1.000000,2.250000
25%,0.020000,6.480000,3.27000,0.380000,-85.022000,5.000000,58.770000
50%,0.050000,20.990000,6.15000,0.525000,1.336000,10.000000,202.335000
75%,0.080000,100.970000,14.36750,0.590000,115.728000,16.000000,798.740000
max,0.210000,6783.020000,164.73000,0.850000,9228.225600,167.000000,45737.330000


# Deal with nans

In [6]:
numeric = ['Discount', 'Unit Price', 'Shipping Cost', 'Product Base Margin', 'Quantity ordered new', 'Sales']
# drop the NaN
df = mm.nandrop(df)
df_num = df[numeric]
# print total samples
print('Total samples: ', df.shape[0])
# print 4-rows and 6-columns
print(df.iloc[:4, :6])

Total samples:  1842
  Order Priority Discount Unit Price Shipping Cost       Ship Mode  \
0           High     0.01       2.84          0.93     Express Air   
1  Not Specified     0.02     500.98            26  Delivery Truck   
2       Critical     0.06       9.48          7.29     Regular Air   
3         Medium     0.09      78.69         19.99     Regular Air   

  Customer Segment  
0        Corporate  
1      Home Office  
2      Home Office  
3   Small Business  


In [7]:
# Get targets
targets = df['Profit']

In [8]:
# list of categorical features
categorical_ = ['Ship Mode', 'Customer Segment', 'Product Category', 'Product Sub-Category', 'Product Container',
        'Country', 'Region', 'State or Province', 'City', 'Order Priority']

# drop the "categorical" features
# drop the profit column
df= df.drop(labels=['Profit'], axis=1)
# drop using 'inplace' which is equivalent to df = df.drop()
data_dropped = df.drop(labels=categorical_, axis=1, inplace=False)
# print partial data
print(data_dropped.iloc[:4,:6])

  Discount Unit Price Shipping Cost Product Base Margin Quantity ordered new  \
0     0.01       2.84          0.93                0.54                    4   
1     0.02     500.98            26                 0.6                   12   
2     0.06       9.48          7.29                0.45                   22   
3     0.09      78.69         19.99                0.43                   16   

     Sales  
0    13.01  
1  6362.85  
2   211.15  
3  1164.45  


In [9]:
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df_num, targets, test_size=0.2, random_state=0)
# Create classifier
lr = LinearRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
lr.score(X_test, y_test)

0.4196702907157779

# Substituting nans with mean

In [10]:
df = pd.read_excel("https://github.com/lifegivesyoulemons/udata/blob/master/SuperStoreUS_2015.xlsx?raw=true", sheet_name = 'Orders', header=0)
df = df.drop(columns=['Row ID', 'Postal Code', 'Order Date', 'Ship Date', 'Order ID', 'Product Name', 'Customer ID', 'Customer Name'])

In [11]:
# turning nans to mean
df_num = df[numeric]
df_num = mm.nantomean(df, numeric)

In [12]:
targets = df['Profit']
targets = mm.nantomean(pd.DataFrame(targets, columns = ['Profit']), ['Profit'])
categorical_ = ['Ship Mode', 'Customer Segment', 'Product Category', 'Product Sub-Category', 'Product Container',
        'Country', 'Region', 'State or Province', 'City', 'Order Priority']
df= df.drop(labels=['Profit'], axis=1)
data_dropped = df.drop(labels=categorical_, axis=1, inplace=False)
print(data_dropped.iloc[:4,:6])

   Discount  Unit Price  Shipping Cost  Product Base Margin  \
0      0.01        2.84           0.93                 0.54   
1      0.02      500.98          26.00                 0.60   
2      0.06        9.48           7.29                 0.45   
3      0.09       78.69          19.99                 0.43   

   Quantity ordered new    Sales  
0                   4.0    13.01  
1                  12.0  6362.85  
2                  22.0   211.15  
3                  16.0  1164.45  


In [13]:
# Split data to train/test sets
X_train, X_test, y_train, y_test = train_test_split(df_num, targets, test_size=0.2, random_state=0)
# Create classifier
lr = LinearRegression()
# Fit classifier
lr = lr.fit(X_train, y_train)
# Predict on test data
y_pred = lr.predict(X_test)
# Get accuracy score
lr.score(X_test, y_test)

0.23691089548565478

### Accuracy is better if we drop nans than if we substitute them to column means

## Standardizer and Scaler in work

In [21]:
np.mean(mm.standardize(df, numeric))

Discount                0.039347
Unit Price              0.428381
Shipping Cost           0.778840
Product Base Margin     0.325682
Quantity ordered new    0.900875
Sales                   0.602533
dtype: float64

In [23]:
np.var(mm.standardize(df, numeric))

Discount                0.043824
Unit Price              5.157042
Shipping Cost           4.009852
Product Base Margin     0.177497
Quantity ordered new    3.600250
Sales                   6.488053
dtype: float64

In [25]:
[np.min(mm.scale(df, numeric)), np.max(mm.scale(df, numeric))]

[Discount                0.000000
 Unit Price              0.000168
 Shipping Cost           0.002679
 Product Base Margin     0.200000
 Quantity ordered new    0.005880
 Sales                   0.000049
 dtype: float64, Discount                0.535500
 Unit Price              1.000000
 Shipping Cost           1.000000
 Product Base Margin     0.919363
 Quantity ordered new    1.000000
 Sales                   1.000000
 dtype: float64]